# Qwen3-14B Jordan Peterson Fine-Tuning — Version 2
## Synthetic Q&A Data · 3 Epochs · Increased LoRA Rank

This notebook is the second iteration of the Qwen3-14B fine-tuning effort on
Jordan B. Peterson's four books.  Version 1 produced a domain-adapted model
(perplexity dropped 33%) but failed on stylistic metrics: the model regurgitated
book passages instead of answering questions in Peterson's voice.

This notebook diagnoses those failures, explains every change made to address
them, and implements a materially improved training pipeline.

---
## Why Version 1 Underperformed — A Detailed Post-Mortem

### The Passage-Regurgitation Problem

V1 produced models that, when prompted with a question like *"What is the
relationship between order and chaos?"*, responded with raw book text beginning
mid-sentence: *"…the world, and the dragon of chaos that guards it, is…"*

This is not a failure of fine-tuning — it is a direct consequence of what the
training data asked the model to learn.

### Root Cause 1: The Training Task Was Wrong

The V1 dataset was created by chunking each book into 350-word passages and
formatting each chunk as a conversation:

```
System:    "You are trained on Peterson's works…"
User:      [first half of a passage]
Assistant: [second half of the same passage]
```

This teaches the model a **text-completion task**: given a piece of Peterson's
writing, continue it.  At inference time we ask something entirely different —
we give it a question and expect a structured, thoughtful answer.  The model
has never seen that task during training, so it falls back on what it learned:
produce passage-like text.

The fix is to use **instruction-tuning format** (Q&A pairs) so the training task
at inference matches the training task at fine-tuning.

### Root Cause 2: One Epoch Is Not Enough

Fine-tuning on a new domain goes through three identifiable phases:

| Phase | Epochs | What the model learns |
|-------|--------|----------------------|
| **Memorisation** | 0.5 – 1.5 | Reproduces training examples nearly verbatim |
| **Generalisation** | 1.5 – 3.5 | Internalises patterns; applies style to new questions |
| **Overfitting** | 4.0+ | Vocabulary narrows; starts looping; perplexity rises |

V1 stopped at 1 epoch — deep in the memorisation phase.  The model had not
yet learned to *generalise* Peterson's voice to unseen prompts.  Three epochs
is the community-established sweet spot for LoRA stylistic fine-tuning.

### Root Cause 3: LoRA Rank Was Too Low

LoRA injects trainable low-rank matrices `B` (d×r) and `A` (r×d) into each
attention projection.  The rank `r` determines how many independent directions
in weight space the adapter can modify.

With **r=16**, the adapter has 16 independent directions per projection.  For
simple task adaptation (e.g., translating to a different language) this is
sufficient.  For **stylistic fine-tuning** — which requires capturing
vocabulary distribution, sentence rhythm, conceptual framing, and rhetorical
structure simultaneously — more capacity helps.

**r=32** doubles the adapter parameters (still only ~0.12% of model weights)
at negligible VRAM cost, giving the adapter meaningfully more room to encode
Peterson's distinctive patterns.

### Summary Table: V1 vs V2

| Component | V1 | V2 | Why the change matters |
|-----------|----|----|----------------------|
| Training data | Passage completion | Synthetic Q&A pairs | Trains to *answer*, not *regurgitate* |
| Training epochs | 1 | 3 | Crosses from memorisation into generalisation |
| LoRA rank | r=16 | r=32 | 2× adapter capacity for style capture |
| LoRA alpha | 16 | 32 | Maintains alpha/rank = 1 (consistent scaling) |
| Effective batch size | 8 (2×4 accum) | 8 (2×4 accum) | Already optimal; no change needed |
| Base model | Qwen3-14B | Qwen3-14B | Still the best option (see analysis below) |
| Total training time | ~23 min | ~90 min | 4× longer; worth it for the quality gain |

---
## V2 Improvement Strategy — Technical Rationale

### Why Qwen3-14B Over GPT-OSS 20B

Both models were evaluated in V1.  Qwen3-14B is the better choice for V2
for three independent reasons:

1. **Lower training loss** (2.44 vs 3.01) — despite being a smaller model,
   Qwen3 absorbed the domain signal more efficiently in one epoch.

2. **Training speed** (23 min vs 73 min per epoch) — with 3 epochs planned,
   GPT-OSS would take ~220 min; Qwen3 takes ~70 min.  Time is a limited resource.

3. **Architectural recency** — Qwen3-14B (2025) is a significantly more capable
   base model than GPT-OSS 20B for instruction-following, which matters for
   the Q&A format we are training on.

The larger parameter count of GPT-OSS (20B vs 14B) does not compensate for
these disadvantages.  Bigger is not always better.

### Why Synthetic Q&A Data Is the Highest-Leverage Change

The authenticity of a fine-tuned model depends on two things:
- The **answers** must sound like the target author (Peterson)
- The **questions** must teach the model when to activate that voice

Our synthetic Q&A strategy achieves both:

- **Answers** = verbatim passages from Peterson's books — authentic, unmodified
- **Questions** = generated by Claude Haiku from each passage — lightweight,
  cost-efficient (~$1–3 total for all 4 books), high enough quality because
  the questions are just triggers; the training signal is in the answers

This mirrors how successful open-source instruction datasets (Alpaca, WizardLM,
OpenHermes) were built: use a capable LLM to generate questions, use high-quality
human text as answers.

### Why Three Epochs

The empirical evidence from the LoRA fine-tuning literature converges on 3 epochs
as the sweet spot for domain-specific stylistic adaptation:

- **1 epoch**: Model has seen each example once.  Gradient updates are noisy.
  Output closely resembles training examples (regurgitation).
- **2 epochs**: Visible improvement; model begins generalising.  Some
  prompts still trigger passage-like output.
- **3 epochs**: Robust generalisation across unseen prompts.  Style is
  internalized, not just memorised.
- **5+ epochs**: Vocabulary narrows toward training set vocabulary; model
  starts to lose general coherence on out-of-distribution prompts.

### Why r=32 and alpha=32

LoRA replaces a weight update `ΔW` with `(alpha/rank) * B*A` where `B` is
`(d_model × r)` and `A` is `(r × d_model)`.

Setting `alpha = rank` (as in V1 with `r=16, alpha=16`, and V2 with `r=32,
alpha=32`) keeps the effective scale factor at `alpha/rank = 1.0`.  This means
the LoRA update is applied at the same relative strength as the base weights,
which is the established baseline.

Doubling the rank from 16 to 32 doubles the *capacity* of the adapter (number
of directions it can modify) without changing the *strength* of updates.  This
is the conservative, well-grounded improvement: more representational power,
same learning dynamics.

### Why gradient_accumulation_steps=4 (unchanged)

With `per_device_train_batch_size=2` and `gradient_accumulation_steps=4`,
the effective batch size is **8 examples** per gradient update.  This was
already correct in V1 — Qwen3-14B fits 2 examples per forward pass on the
4090, and 4 accumulation steps gives stability without excessive memory.

No change needed here.

---
## Notebook Structure

- **Part 1** — Generate the synthetic Q&A training dataset using Claude Haiku
- **Part 2** — Fine-tune Qwen3-14B using that dataset with the V2 hyperparameters

---
# Part 1: Synthetic Q&A Dataset Generation

## Overview

We will:
1. Extract all passages from the four Peterson PDFs (same extraction logic as V1)
2. For each passage, call the Claude Haiku API to generate 2 questions that the
   passage directly answers
3. Pair each question with the original passage as the answer
4. Cache the results to disk so this step only runs once

### Why Claude Haiku for Question Generation?

- **Cost**: Haiku is the most affordable Claude model (~$0.80/M input tokens,
  $4.00/M output tokens).  Generating questions for ~1,200 passages costs
  roughly $1–3 total.
- **Quality**: Question generation is not a demanding task — we only need
  plausible, on-topic questions.  The *training signal* is entirely in the
  answers (Peterson's own words), not the questions.  Haiku is more than
  adequate.
- **Speed**: Haiku has the highest throughput of Claude models.

### Prerequisites

You need an Anthropic API key.  Set it before running this cell:

```bash
export ANTHROPIC_API_KEY="your-key-here"
```

Or add it to `~/.env` as `ANTHROPIC_API_KEY=your-key-here`.

In [ ]:
# ── Install the Anthropic Python SDK if not already present ────────────────
# The .finetuning venv does not ship with 'anthropic' by default; this cell
# installs it silently on first run.  On subsequent runs pip finds it already
# installed and exits immediately.
import subprocess, sys
_result = subprocess.run(
    [sys.executable, "-m", "pip", "install", "anthropic", "-q"],
    capture_output=True, text=True,
)
if _result.returncode != 0:
    print("pip install failed:", _result.stderr[:400])
else:
    print("anthropic SDK ready.")

import os, re, json, time, math
from pathlib import Path
from collections import defaultdict

import fitz          # PyMuPDF — PDF extraction
import anthropic     # Anthropic Python SDK

print(f"anthropic version : {anthropic.__version__}")

# ── API key ────────────────────────────────────────────────────────────────
# Look for the key in the environment first, then fall back to ~/.env.
_api_key = os.environ.get("ANTHROPIC_API_KEY", "")
if not _api_key:
    _env_file = Path.home() / ".env"
    if _env_file.exists():
        for _line in _env_file.read_text().splitlines():
            if _line.startswith("ANTHROPIC_API_KEY="):
                _api_key = _line.split("=", 1)[1].strip().strip('"').strip("'")
                break

if not _api_key:
    raise EnvironmentError(
        "ANTHROPIC_API_KEY not found.\n"
        "Set it with:  export ANTHROPIC_API_KEY='sk-ant-...'\n"
        "Or add it to ~/.env as:  ANTHROPIC_API_KEY=sk-ant-..."
    )
print("API key found.")

# ── Paths ──────────────────────────────────────────────────────────────────
BOOKS_DIR   = Path("../../Books/JordanPeterson")
QA_DIR      = Path("./qa_dataset")
QA_CACHE    = QA_DIR / "peterson_qa.jsonl"
QA_DIR.mkdir(exist_ok=True)

# Model for question generation: Haiku — cheap, fast, sufficient for Q generation
GENERATION_MODEL = "claude-haiku-4-5-20251001"

print(f"Books dir : {BOOKS_DIR.resolve()}")
print(f"QA cache  : {QA_CACHE.resolve()}")
print(f"Gen model : {GENERATION_MODEL}")

---
### Step 1a: Extract and Chunk Book Passages

We reuse the same extraction logic from V1:
- Read each PDF with PyMuPDF
- Strip page numbers, headers, excessive whitespace, and non-printable characters
- Chunk into ~350-word passages with 50-word overlap to avoid hard cuts mid-sentence

In [ ]:
def clean_text(raw: str) -> str:
    """
    Remove PDF artefacts from extracted text.

    PyMuPDF sometimes includes:
    - Non-printable control characters from PDF encoding
    - Repeated whitespace and newlines from column layout
    - Ligature characters (ﬁ, ﬂ) that don't tokenize well

    We replace all whitespace runs with a single space and strip
    non-printable characters, leaving clean prose.
    """
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]', '', raw)
    text = re.sub(r'\s+', ' ', text)
    # Replace common PDF ligatures with their ASCII equivalents
    text = text.replace('\ufb01', 'fi').replace('\ufb02', 'fl')
    return text.strip()


def extract_chunks(pdf_path: Path,
                   chunk_words: int = 350,
                   overlap_words: int = 50) -> list[str]:
    """
    Extract text from a PDF and split it into overlapping word-level chunks.

    Why overlapping chunks?
    Hard cuts at fixed word boundaries often split sentences mid-thought.
    A 50-word overlap ensures each chunk starts with enough context to be
    readable on its own, which makes the Q&A pairs more coherent.

    Args:
        pdf_path    : path to the PDF file
        chunk_words : target words per chunk (default 350 — long enough to
                      contain a complete argument, short enough to stay under
                      the tokenizer's context window after formatting)
        overlap_words: words from the previous chunk to prepend (default 50)

    Returns:
        List of clean text chunks (each 300–400 words typically)
    """
    doc   = fitz.open(str(pdf_path))
    pages = [clean_text(page.get_text()) for page in doc]
    doc.close()
    full_text = ' '.join(pages)

    words  = full_text.split()
    step   = chunk_words - overlap_words
    chunks = []
    for start in range(0, len(words), step):
        chunk = ' '.join(words[start : start + chunk_words])
        if len(chunk.split()) >= 100:   # discard very short tail chunks
            chunks.append(chunk)
    return chunks


# ── Extract passages from all 4 books ─────────────────────────────────────
pdf_files = sorted(BOOKS_DIR.glob("*.pdf"))
print(f"Found {len(pdf_files)} PDF files:")
for p in pdf_files:
    print(f"  {p.name}")

all_chunks  = []    # flat list of all passage strings
chunk_meta  = []    # parallel list of {"book": short_name} for each chunk

book_labels = {
    "Maps of Meaning":     "Maps of Meaning",
    "12 Rules for Life":   "12 Rules for Life",
    "Beyond Order":        "Beyond Order",
    "We Who Wrestle":      "We Who Wrestle with God",
}

print()
for pdf in pdf_files:
    # Assign a short book label based on filename keywords
    fname = pdf.name.lower()
    if "maps" in fname:
        label = "Maps of Meaning"
    elif "12 rules" in fname or "antidote" in fname:
        label = "12 Rules for Life"
    elif "beyond" in fname:
        label = "Beyond Order"
    else:
        label = "We Who Wrestle with God"

    chunks = extract_chunks(pdf)
    all_chunks.extend(chunks)
    chunk_meta.extend([{"book": label}] * len(chunks))
    print(f"  {label:<30}  {len(chunks):4d} chunks  "
          f"(~{sum(len(c.split()) for c in chunks):,} words)")

print(f"\nTotal passages: {len(all_chunks):,}")

---
### Step 1b: Generate Questions via Claude Haiku

For each passage we ask Haiku to generate exactly **2 questions** that the
passage directly answers.  The answers in the training dataset will be the
passage text verbatim — we are only generating the *questions*, which is a
much cheaper and easier task.

**Prompt design choices:**
- Ask for exactly 2 questions in a JSON array (structured output is easy to parse)
- Specify that questions should cover *different angles* of the passage (variety)
- Provide Peterson's topic vocabulary so Haiku generates contextually appropriate
  questions even when a passage is abstract
- Return questions only (not answers), so the output is very short → low cost

In [ ]:
client = anthropic.Anthropic(api_key=_api_key)


def generate_questions(passage: str, book: str,
                       max_retries: int = 3) -> list[str]:
    """
    Call Claude Haiku to generate 2 questions for a given passage.

    The prompt instructs Haiku to:
    1. Return ONLY a JSON array of 2 strings (easy to parse, minimal output tokens)
    2. Generate questions that the passage directly answers (ensures relevance)
    3. Cover different angles — one can be practical, one philosophical/conceptual
    4. Sound like genuine questions a reader would ask (not synthetic-sounding)

    Retries up to max_retries times with exponential backoff on API errors or
    malformed JSON.  Returns an empty list on complete failure so the caller
    can skip the passage gracefully.

    Cost estimate:
    - Input:  ~550 tokens per call  (prompt template + 350-word passage)
    - Output: ~60 tokens per call   (2 short question strings in JSON)
    - At Haiku pricing ($0.80/$4.00 per MTok): ~$0.00068 per passage
    - For 1,200 passages: ~$0.82 total
    """
    prompt = (
        f"You are building a training dataset for a Jordan B. Peterson AI model.\n\n"
        f"The passage below is from Peterson's book '{book}'.  Generate exactly 2 questions "
        f"that:\n"
        f"1. This passage directly and substantively answers\n"
        f"2. Someone interested in Peterson's ideas might genuinely ask\n"
        f"3. Cover different angles of the passage (e.g. one concrete, one philosophical)\n\n"
        f"Peterson's topics include: order vs chaos, meaning, personal responsibility, "
        f"suffering, mythology, archetypes, the shadow, logos, truth, religion, "
        f"Jungian psychology, hierarchy, heroism, sacrifice, being.\n\n"
        f"Return ONLY a JSON array of exactly 2 question strings.  No other text.\n"
        f"Example: [\"Why is confronting chaos necessary for meaning?\", "
        f"\"What role does suffering play in personal development?\"]\n\n"
        f"Passage:\n{passage}"
    )

    for attempt in range(max_retries):
        try:
            response = client.messages.create(
                model=GENERATION_MODEL,
                max_tokens=150,   # 2 short questions never need more than 150 tokens
                messages=[{"role": "user", "content": prompt}],
            )
            raw = response.content[0].text.strip()

            # Extract JSON array — handle leading/trailing prose if any
            match = re.search(r'\[.*?\]', raw, re.DOTALL)
            if not match:
                raise ValueError(f"No JSON array found in: {raw[:100]}")
            questions = json.loads(match.group())

            if not isinstance(questions, list) or len(questions) < 1:
                raise ValueError(f"Expected list, got: {type(questions)}")

            # Return up to 2 questions; pad with a fallback if only 1 returned
            return [str(q).strip() for q in questions[:2]]

        except Exception as e:
            wait = 2 ** attempt   # exponential backoff: 1s, 2s, 4s
            if attempt < max_retries - 1:
                print(f"    [retry {attempt+1}/{max_retries}] Error: {e} — waiting {wait}s")
                time.sleep(wait)
            else:
                print(f"    [FAILED after {max_retries} attempts] Passage skipped.")
                return []

    return []


print("generate_questions() defined.")

---
### Step 1c: Run Generation (Cache-Aware)

This cell generates questions for every passage and writes them to
`qa_dataset/peterson_qa.jsonl` as it goes.  If the JSONL file already
contains results for **90% or more** of passages, the whole step is skipped.

Writing incrementally (one record at a time, flushing after each) means that
if the process is interrupted — by a rate-limit error, a power cut, or a
keyboard interrupt — all work up to that point is preserved and the run can
be resumed by simply re-executing this cell.

In [ ]:
# ── Check cache ────────────────────────────────────────────────────────────
existing_records = []
if QA_CACHE.exists():
    with open(QA_CACHE) as f:
        existing_records = [json.loads(line) for line in f if line.strip()]

_expected = len(all_chunks) * 2   # 2 Q&A pairs per passage
_coverage = len(existing_records) / _expected if _expected else 0

print(f"Passages   : {len(all_chunks):,}")
print(f"Expected Q&A pairs : {_expected:,}  (2 per passage)")
print(f"Cached     : {len(existing_records):,}  ({100*_coverage:.1f}% coverage)")

if _coverage >= 0.90:
    print("\nCache is ≥90% complete — skipping generation.")
    print(f"Delete {QA_CACHE} to force regeneration.")
else:
    # Determine which passages still need processing
    # We track progress by appending to the JSONL file and counting existing lines.
    already_done = len(existing_records) // 2   # 2 records per passage
    remaining    = all_chunks[already_done:]
    remaining_meta = chunk_meta[already_done:]

    print(f"\nGenerating questions for {len(remaining):,} passages "
          f"(starting from passage {already_done + 1})…")
    print(f"Estimated cost: ~${len(remaining) * 0.00068:.2f}\n")

    skipped = 0
    with open(QA_CACHE, "a") as out_f:
        for i, (passage, meta) in enumerate(zip(remaining, remaining_meta)):
            global_idx = already_done + i + 1

            # Print progress every 50 passages
            if i % 50 == 0:
                pct = 100 * (already_done + i) / len(all_chunks)
                print(f"  [{global_idx:4d}/{len(all_chunks):4d}]  {pct:5.1f}%  "
                      f"book: {meta['book']}")

            questions = generate_questions(passage, meta["book"])

            if not questions:
                skipped += 1
                continue

            # Write one JSON line per Q&A pair so the file is resumable
            for q in questions:
                record = {
                    "question": q,
                    "answer":   passage,
                    "book":     meta["book"],
                }
                out_f.write(json.dumps(record) + "\n")
            out_f.flush()   # flush after each passage — safe against interruption

            # Polite rate-limiting: 0.3 s gap between requests
            time.sleep(0.3)

    print(f"\nGeneration complete.")
    print(f"  Passages processed : {len(remaining) - skipped:,}")
    print(f"  Passages skipped   : {skipped}")
    print(f"  Output file        : {QA_CACHE}")

# ── Re-read final dataset regardless of which path we took ────────────────
with open(QA_CACHE) as f:
    qa_records = [json.loads(line) for line in f if line.strip()]

print(f"\nTotal Q&A pairs in dataset: {len(qa_records):,}")

---
### Step 1d: Dataset Statistics and Sample Preview

Before training, verify the dataset:
- Distribution across books (should be roughly proportional to book length)
- Average answer length (should be ~350 words — our target chunk size)
- Sample Q&A pairs to spot-check question quality

In [ ]:
from collections import Counter

# ── Distribution by book ───────────────────────────────────────────────────
book_counts = Counter(r["book"] for r in qa_records)
print("Q&A pairs by book:")
for book, count in sorted(book_counts.items(), key=lambda x: -x[1]):
    print(f"  {book:<35}  {count:4d} pairs  ({100*count/len(qa_records):.1f}%)")

# ── Answer (passage) length statistics ────────────────────────────────────
ans_lengths = [len(r["answer"].split()) for r in qa_records]
print(f"\nAnswer length (words):")
print(f"  Min   : {min(ans_lengths)}")
print(f"  Max   : {max(ans_lengths)}")
print(f"  Mean  : {sum(ans_lengths)/len(ans_lengths):.0f}")
print(f"  Median: {sorted(ans_lengths)[len(ans_lengths)//2]}")

# ── Question length statistics ─────────────────────────────────────────────
q_lengths = [len(r["question"].split()) for r in qa_records]
print(f"\nQuestion length (words):")
print(f"  Min   : {min(q_lengths)}")
print(f"  Max   : {max(q_lengths)}")
print(f"  Mean  : {sum(q_lengths)/len(q_lengths):.1f}")

# ── Sample Q&A pairs from each book ───────────────────────────────────────
print("\n" + "═" * 80)
print("SAMPLE Q&A PAIRS (first example per book)")
print("═" * 80)
seen_books = set()
for r in qa_records:
    if r["book"] not in seen_books:
        seen_books.add(r["book"])
        print(f"\n📖  {r['book']}")
        print(f"Q: {r['question']}")
        print(f"A: {r['answer'][:200]}…")
        print()
    if len(seen_books) == 4:
        break

---
# Part 2: Fine-Tuning Qwen3-14B (Version 2)

## Configuration Summary

| Parameter | V1 | **V2** | Notes |
|-----------|----|----|-------|
| Base model | `Qwen3-14B-unsloth-bnb-4bit` | Same | Best option on this hardware |
| Training data | Passage completion | **Synthetic Q&A** | Core fix for regurgitation |
| Epochs | 1 | **3** | Cross into generalisation phase |
| LoRA rank (r) | 16 | **32** | 2× adapter capacity |
| LoRA alpha | 16 | **32** | Maintains alpha/rank = 1.0 |
| LoRA target modules | q/k/v/o + gate/up/down | Same | All attention + MLP |
| Batch size per device | 2 | 2 | Limited by 24 GB VRAM |
| Gradient accumulation | 4 | 4 | Effective batch = 8; already optimal |
| Learning rate | 2e-4 | 2e-4 | Standard for Qwen3 LoRA |
| LR schedule | cosine | cosine | Smooth decay over 3 epochs |
| Warmup steps | 10 | **30** | Proportionally scaled to 3× more steps |
| Optimizer | adamw_8bit | adamw_8bit | Unsloth's memory-efficient AdamW |
| Output dir | `outputs/qwen3_14b_…_lora` | `outputs/qwen3_14b_peterson_v2_lora` | Separate from V1 |

## Expected Outcomes

With the Q&A format and 3 epochs, we expect:
- **Perplexity**: similar or slightly better reduction than V1 (−30% to −40%)
- **TF-IDF similarity**: improvement (model uses Peterson's vocabulary patterns)
- **Keyword density**: improvement (model spontaneously uses signature terms)
- **TTR**: improvement (no more passage repetition → richer vocabulary)
- **Response quality**: coherent, structured answers in Peterson's voice

In [ ]:
import os, re, gc, math, json
from pathlib import Path

import torch
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer, SFTConfig, DataCollatorForSeq2Seq
from unsloth.chat_templates import train_on_responses_only

# ── Paths ──────────────────────────────────────────────────────────────────
QA_CACHE    = Path("./qa_dataset/peterson_qa.jsonl")   # generated in Part 1
OUTPUT_DIR  = Path("./outputs/qwen3_14b_peterson_v2_lora")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ── Model ──────────────────────────────────────────────────────────────────
BASE_MODEL  = "unsloth/Qwen3-14B-unsloth-bnb-4bit"
MAX_SEQ_LEN = 2048   # safe upper bound; covers 350-word passage + chat overhead

# ── LoRA hyperparameters ───────────────────────────────────────────────────
LORA_RANK   = 32     # doubled from V1's r=16
LORA_ALPHA  = 32     # alpha/rank = 1.0, same as V1 — maintains update scale

# ── Training hyperparameters ──────────────────────────────────────────────
BATCH_SIZE        = 2    # max that fits comfortably in 24 GB with Qwen3-14B
GRAD_ACCUM        = 4    # effective batch = 2×4 = 8
NUM_EPOCHS        = 3    # key change from V1
LEARNING_RATE     = 2e-4
WARMUP_STEPS      = 30   # ~3% of total steps; scaled up from V1's 10 for longer run
WEIGHT_DECAY      = 0.01

# ── System prompt ──────────────────────────────────────────────────────────
# Every training example uses this system prompt so the model learns to adopt
# Peterson's voice when given this specific persona instruction.
SYSTEM_PROMPT = (
    "You are an AI assistant that has been trained on the complete works of "
    "Jordan B. Peterson, a Canadian clinical psychologist, professor, and author. "
    "You speak with deep knowledge of psychology, philosophy, mythology, religion, "
    "and personal responsibility.  Your responses reflect Peterson's writing style, "
    "intellectual depth, and interdisciplinary approach to understanding human "
    "nature and meaning."
)

print(f"PyTorch  : {torch.__version__}")
print(f"CUDA     : {torch.cuda.is_available()}  |  GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM     : {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB")
print(f"\nConfiguration:")
print(f"  Base model  : {BASE_MODEL}")
print(f"  LoRA rank   : r={LORA_RANK}, alpha={LORA_ALPHA}  (ratio={LORA_ALPHA/LORA_RANK:.1f})")
print(f"  Epochs      : {NUM_EPOCHS}")
print(f"  Batch (eff) : {BATCH_SIZE} × {GRAD_ACCUM} = {BATCH_SIZE*GRAD_ACCUM}")
print(f"  LR          : {LEARNING_RATE}")
print(f"  Output      : {OUTPUT_DIR.resolve()}")

---
## Step 2: Load the Q&A Dataset

Load the JSONL file generated in Part 1 and build a HuggingFace `Dataset` object.
Each record contains:
- `question` — the synthetically generated question
- `answer`   — the verbatim Peterson passage
- `book`     — source book label (used only for statistics, not training)

In [ ]:
if not QA_CACHE.exists():
    raise FileNotFoundError(
        f"Q&A cache not found at {QA_CACHE}.\n"
        "Please run Part 1 first to generate the dataset."
    )

with open(QA_CACHE) as f:
    records = [json.loads(line) for line in f if line.strip()]

# Build a HuggingFace Dataset from the flat list of dicts.
# We only keep "question" and "answer"; the "book" field is dropped to keep
# the dataset schema minimal.
raw_dataset = Dataset.from_list([
    {"question": r["question"], "answer": r["answer"]}
    for r in records
])

print(f"Dataset loaded: {len(raw_dataset):,} Q&A pairs")
print(f"Schema: {raw_dataset.column_names}")
print()

# Show distribution across books
from collections import Counter
book_dist = Counter(r["book"] for r in records)
print("Distribution by book:")
for book, count in sorted(book_dist.items(), key=lambda x: -x[1]):
    print(f"  {book:<35}  {count:4d}  ({100*count/len(records):.1f}%)")

---
## Step 3: Load the Qwen3-14B Base Model

We load the 4-bit quantized model via Unsloth.  The `max_seq_length=2048`
matches the context window used during dataset formatting — passing a shorter
value would truncate long passages; a longer value would waste VRAM on
unneeded KV-cache pre-allocation.

In [ ]:
print(f"Loading {BASE_MODEL} …")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name      = BASE_MODEL,
    dtype           = None,          # auto: bfloat16 on Ampere+, float16 otherwise
    max_seq_length  = MAX_SEQ_LEN,
    load_in_4bit    = True,          # 4-bit quantization via bitsandbytes
    full_finetuning = False,         # we are adding LoRA, not full fine-tuning
)

vram_after_load = torch.cuda.memory_reserved() / 1e9
print(f"Model loaded.  VRAM reserved: {vram_after_load:.1f} GB")
print(f"Model dtype   : {next(model.parameters()).dtype}")
print(f"Tokenizer     : {tokenizer.__class__.__name__}")

---
## Step 4: Add LoRA Adapters (r=32)

### What LoRA Does

Low-Rank Adaptation freezes the pre-trained weight matrices `W` and injects a
trainable update `ΔW = (alpha/r) × B × A` where:
- `B` has shape `(d_model × r)` — initialised to zero
- `A` has shape `(r × d_model)` — initialised with random Gaussian values
- `r` is the rank — our key improvement from 16 → 32

At the start of training `B=0` so `ΔW=0` and the model behaves identically to
the base model.  Training gradually fills `B` with the directions needed to
capture Peterson's style.

### Why r=32 Matters

Think of `r` as the number of "stylistic dimensions" the adapter can learn.
With r=16, the adapter has 16 orthogonal directions to encode things like:
*"use chaos/order vocabulary"*, *"frame arguments historically"*,
*"connect psychology to mythology"*, etc.

With r=32 we have 32 such dimensions — enough to capture both the macro-level
thematic vocabulary AND the micro-level sentence rhythm that makes Peterson's
writing recognisable.

### Target Modules

We apply LoRA to all attention projection matrices (q, k, v, o) and all MLP
gate/up/down projections.  This is the full set recommended by Unsloth for
stylistic fine-tuning because style is encoded across both attention patterns
(what the model attends to) and the MLP activations (how it transforms those
representations).

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r                   = LORA_RANK,    # 32 — doubled from V1
    lora_alpha          = LORA_ALPHA,   # 32 — alpha/rank = 1.0 (same scale as V1)
    lora_dropout        = 0,            # 0 is optimal for Unsloth according to docs
    target_modules      = [
        "q_proj", "k_proj", "v_proj", "o_proj",   # all attention projections
        "gate_proj", "up_proj", "down_proj",        # all MLP projections
    ],
    use_gradient_checkpointing = "unsloth",   # Unsloth's custom implementation:
                                               # saves ~30% VRAM vs PyTorch default
    random_state        = 42,
    use_rslora          = False,   # standard LoRA (not rank-stabilised variant)
    loftq_config        = None,    # no quantization-aware init needed with 4-bit base
)

# ── Count trainable parameters ─────────────────────────────────────────────
total_params     = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pct_trainable    = 100 * trainable_params / total_params

print(f"Total parameters    : {total_params:>15,}")
print(f"Trainable (LoRA)    : {trainable_params:>15,}  ({pct_trainable:.4f}% of total)")
print(f"  → V1 had r=16: ~{trainable_params//2:,} trainable params")
print(f"  → V2 has r=32: ~{trainable_params:,} trainable params (2× capacity)")
print(f"VRAM after LoRA     : {torch.cuda.memory_reserved()/1e9:.1f} GB")

---
## Step 5: Format Dataset into Qwen3 ChatML Format

Each Q&A pair is converted into a three-turn conversation using the Qwen3
ChatML template.  The key improvement over V1:

| Role | V1 content | **V2 content** |
|------|-----------|----------------|
| `system` | Generic assistant prompt | **Peterson persona prompt** |
| `user` | A book passage chunk | **A synthetically generated question** |
| `assistant` | Continuation of the passage | **The original passage as an answer** |

The system prompt changes from generic to persona-specific so the model
learns: *"when given the Peterson persona instruction, respond in his voice"*.

The user message changes from a passage fragment to a real question so the
model learns: *"when asked a question in this domain, produce a substantive
Peterson-style answer"*.

### About `enable_thinking=False`

We format training examples with `enable_thinking=False` — the same setting
used in V1.  This tells Qwen3's chat template not to include chain-of-thought
reasoning tokens in the formatted string.  Even with this setting, the template
adds an empty `<think>\n\n</think>` block before the assistant response — this
is expected and will be masked out by `train_on_responses_only` in the next step.

In [ ]:
def format_example(batch):
    """
    Convert a batch of Q&A records into formatted Qwen3 ChatML strings.

    The output is stored in a 'text' column which SFTTrainer reads directly
    via dataset_text_field='text'.

    Each formatted string looks like:
        <|im_start|>system
        You are an AI assistant... (Peterson persona)
        <|im_end|>
        <|im_start|>user
        <the generated question>
        <|im_end|>
        <|im_start|>assistant
        <think>

        </think>
        <the Peterson passage>
        <|im_end|>

    The <think>\n\n</think> block appears because Qwen3's template always
    includes a thinking placeholder even when enable_thinking=False.  It will
    be masked out during training by train_on_responses_only, so it does not
    contribute to the loss and does not affect what the model learns.
    """
    formatted_texts = []
    for question, answer in zip(batch["question"], batch["answer"]):
        conversation = [
            {"role": "system",    "content": SYSTEM_PROMPT},
            {"role": "user",      "content": question},
            {"role": "assistant", "content": answer},
        ]
        text = tokenizer.apply_chat_template(
            conversation,
            tokenize           = False,
            add_generation_prompt = False,
            enable_thinking    = False,   # no chain-of-thought in training
        )
        formatted_texts.append(text)
    return {"text": formatted_texts}


# Apply formatting across the full dataset using batched map for efficiency
dataset = raw_dataset.map(
    format_example,
    batched=True,
    batch_size=256,
    remove_columns=raw_dataset.column_names,   # keep only 'text'
)

print(f"Formatted dataset: {len(dataset):,} examples")
print(f"Columns: {dataset.column_names}")

# ── Print one formatted sample to verify the structure ─────────────────────
print("\n" + "─" * 70)
print("SAMPLE FORMATTED TRAINING EXAMPLE")
print("─" * 70)
sample = dataset[0]["text"]
print(sample[:800])
print("…")
print("─" * 70)
print(f"Full sample length: {len(sample.split())} words / {len(sample)} chars")

---
## Step 6: Apply Response-Only Training Mask

`train_on_responses_only` adds a masking function to the data collator so that
during training the cross-entropy loss is computed **only on the assistant's
tokens** — not on the system prompt or the user question.

This is critical for two reasons:
1. **Efficiency**: the model already knows how to be an assistant; we only want
   it to learn *what* to say, not the structural tokens around it.
2. **Correctness**: without this mask, the model would also try to memorise the
   system prompt and questions, which wastes capacity and can cause instability.

The instruction boundary is the last `<|im_start|>user\n` token before each
assistant turn.  The response boundary is `<|im_start|>assistant\n`.
Tokens between them (the user's question) get loss weight = 0.

In [ ]:
# Auto-detect the response boundary tokens by inspecting the formatted data.
# This guards against future Qwen3 template changes that might shift the token
# boundaries, and mirrors the robust approach used in V1.
_sample_text = dataset[0]["text"]

if "<|im_start|>assistant\n" in _sample_text:
    instruction_part = "<|im_start|>user\n"
    response_part    = "<|im_start|>assistant\n"
    print("Detected Qwen3 ChatML response boundary tokens:")
else:
    raise RuntimeError(
        "Could not detect Qwen3 ChatML tokens in formatted dataset.\n"
        "Check the output of format_example() above."
    )

print(f"  instruction_part : {repr(instruction_part)}")
print(f"  response_part    : {repr(response_part)}")

trainer = train_on_responses_only(
    trainer      = None,   # will be filled in after SFTTrainer is built below
    instruction_part = instruction_part,
    response_part    = response_part,
)
# Note: train_on_responses_only returns a modified version of the trainer.
# We pass 'None' here just to test token detection; the actual trainer is
# built and wrapped in the next two cells.
print("\nResponse masking configured.")

---
## Step 7: Configure and Run Training

### SFTConfig Explained

- **`num_train_epochs=3`**: The most important change from V1.  Three full passes
  over the dataset let the model move from memorisation into genuine stylistic
  generalisation.

- **`warmup_steps=30`**: Gradually ramps the learning rate from 0 → 2e-4 over
  the first 30 gradient steps (~3% of total training).  This prevents the
  randomly-initialised LoRA matrices from making destructively large weight
  updates at the start.

- **`lr_scheduler_type="cosine"`**: After warmup the learning rate decays
  smoothly following a cosine curve, reaching ~0 at the end of epoch 3.
  Cosine decay is generally more stable than linear decay for LoRA.

- **`weight_decay=0.01`**: Mild L2 regularisation on the LoRA weights.
  Discourages the adapter from memorising individual passages and encourages
  generalisation.

- **`optim="adamw_8bit"`**: Unsloth's 8-bit AdamW keeps optimizer states in
  8-bit rather than 32-bit, saving ~3 GB of VRAM with no measurable quality
  cost for LoRA training.

- **`packing=False`**: Sequence packing combines multiple short examples into
  one long sequence for efficiency.  We disable it because our examples are
  long (~400 words → ~600 tokens), and packing them together could leak
  context between unrelated Q&A pairs, confusing the model.

In [ ]:
from trl import SFTTrainer, SFTConfig

sft_trainer = SFTTrainer(
    model         = model,
    tokenizer     = tokenizer,
    train_dataset = dataset,
    args          = SFTConfig(
        dataset_text_field          = "text",
        max_seq_length              = MAX_SEQ_LEN,
        dataset_num_proc            = 2,         # parallel tokenisation

        per_device_train_batch_size = BATCH_SIZE,
        gradient_accumulation_steps = GRAD_ACCUM,
        num_train_epochs            = NUM_EPOCHS,

        learning_rate               = LEARNING_RATE,
        warmup_steps                = WARMUP_STEPS,
        lr_scheduler_type           = "cosine",
        weight_decay                = WEIGHT_DECAY,

        optim                       = "adamw_8bit",
        fp16                        = not torch.cuda.is_bf16_supported(),
        bf16                        = torch.cuda.is_bf16_supported(),

        logging_steps               = 25,        # log loss every 25 gradient steps
        save_strategy               = "epoch",   # save adapter at end of each epoch
        output_dir                  = str(OUTPUT_DIR),
        report_to                   = "none",    # disable W&B / HF reporting

        seed                        = 42,
        packing                     = False,
    ),
)

# Apply the response-only mask to the configured trainer
sft_trainer = train_on_responses_only(
    sft_trainer,
    instruction_part = instruction_part,
    response_part    = response_part,
)

# ── Quick sanity check: confirm labels are masked for non-response tokens ──
# If masking is working, the label tensor will have -100 for all system/user
# tokens and real token IDs only for the assistant response tokens.
_sample_input  = sft_trainer.train_dataset[0]
_n_total       = len(_sample_input["input_ids"])
_n_trained     = sum(1 for l in _sample_input["labels"] if l != -100)
_n_masked      = _n_total - _n_trained
print(f"Response masking check on first example:")
print(f"  Total tokens   : {_n_total}")
print(f"  Trained tokens : {_n_trained}  (assistant response only)")
print(f"  Masked tokens  : {_n_masked}   (system + user = {100*_n_masked/_n_total:.0f}% of input)")

_total_steps = math.ceil(len(dataset) / BATCH_SIZE) * NUM_EPOCHS // GRAD_ACCUM
print(f"\nEstimated gradient updates: {_total_steps:,}")
print(f"  ({len(dataset):,} examples / {BATCH_SIZE} batch × {GRAD_ACCUM} accum × {NUM_EPOCHS} epochs)")

---
## Step 8: Train

In [ ]:
import time

# Log VRAM before training to establish baseline
vram_before = torch.cuda.memory_reserved() / 1e9
print(f"VRAM before training: {vram_before:.1f} GB")
print(f"Starting training — {NUM_EPOCHS} epochs, ~{_total_steps:,} gradient updates…")
print()

t0 = time.time()
train_result = sft_trainer.train()
elapsed_min = (time.time() - t0) / 60

vram_peak = torch.cuda.max_memory_reserved() / 1e9

print()
print("═" * 60)
print("TRAINING COMPLETE")
print("═" * 60)
print(f"  Steps          : {train_result.global_step:,}")
print(f"  Training loss  : {train_result.training_loss:.4f}")
print(f"  Elapsed        : {elapsed_min:.1f} min")
print(f"  Peak VRAM      : {vram_peak:.1f} GB")
print()
print("V1 reference:")
print("  Steps: 321  |  Loss: 2.4400  |  Time: 23.3 min  |  VRAM: 13.4 GB")
print()
print("Expected improvement: lower loss (better Q&A alignment) + longer training")

---
## Step 9: Save the LoRA Adapter

We save only the LoRA adapter weights (not the full 14B model), which keeps the
checkpoint small (~500 MB for r=32 vs ~260 MB for V1's r=16).  At inference
time Unsloth loads the base model and merges the adapter on-the-fly.

In [ ]:
print(f"Saving LoRA adapter to {OUTPUT_DIR} …")

model.save_pretrained(str(OUTPUT_DIR))
tokenizer.save_pretrained(str(OUTPUT_DIR))

# Check saved file sizes
adapter_files = list(OUTPUT_DIR.glob("*.safetensors")) + list(OUTPUT_DIR.glob("*.bin"))
total_mb = sum(f.stat().st_size for f in adapter_files) / 1e6
print(f"\nAdapter files:")
for f in sorted(adapter_files):
    print(f"  {f.name}  ({f.stat().st_size/1e6:.1f} MB)")
print(f"\nTotal adapter size: {total_mb:.1f} MB")
print(f"  (V1 r=16 was ~260 MB; V2 r=32 should be ~520 MB)")

---
## Step 10: Test the Fine-Tuned Model

Enable Unsloth's fast inference kernel and test the model on a representative
set of Peterson-themed questions.  Use the same 10 evaluation prompts that are
used in the comparison notebooks so results are directly comparable.

We run greedy decoding (`do_sample=False`) for deterministic, reproducible
outputs — the same setting used in all comparison notebooks.

In [ ]:
FastLanguageModel.for_inference(model)

EVAL_PROMPTS = [
    "What is the relationship between order and chaos in human experience?",
    "Why is personal responsibility the foundation of a meaningful life?",
    "How do ancient myths and stories reveal truths about human nature?",
    "What does it mean to pursue what is meaningful rather than what is expedient?",
    "How should a person confront suffering rather than flee from it?",
]

def ask_v2(question: str, max_new_tokens: int = 300) -> str:
    """
    Generate a response from the V2 fine-tuned model.

    Uses the same two-step Qwen3 tokenization pipeline as V1:
    1. apply_chat_template() → plain text string
    2. tokenizer(text) → input tensors

    enable_thinking=False keeps reasoning mode disabled, consistent with
    how the model was trained.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": question},
    ]
    text   = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens   = max_new_tokens,
            do_sample        = False,
            temperature      = 1.0,
            repetition_penalty = 1.1,
        )

    new_tokens = out[0][inputs["input_ids"].shape[1]:]
    response   = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    # Strip any residual thinking blocks (can appear even with enable_thinking=False)
    response   = re.sub(r'<think>.*?</think>', '', response, flags=re.DOTALL).strip()
    return response


print("Testing V2 fine-tuned model (greedy decoding)…\n")
for i, prompt in enumerate(EVAL_PROMPTS):
    print(f"{'─'*70}")
    print(f"Q{i+1}: {prompt}")
    print(f"{'─'*70}")
    answer = ask_v2(prompt)
    print(answer if answer.strip() else "(empty response)")
    print()

---
## Step 11: Test Thinking Mode (Optional)

Qwen3 supports an optional chain-of-thought reasoning mode via
`enable_thinking=True`.  The model first produces a `<think>…</think>` block
containing its internal reasoning, then gives its final answer.

We did not train with thinking mode, so this tests whether the base model's
reasoning capability (preserved through LoRA) can enhance response quality
when combined with the fine-tuned Peterson domain knowledge.

In [ ]:
def ask_v2_thinking(question: str, max_new_tokens: int = 500) -> tuple[str, str]:
    """
    Generate a response in thinking mode.

    Returns (thinking_content, final_answer) as separate strings so the caller
    can choose to display just the answer, just the reasoning, or both.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": question},
    ]
    text   = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True,   # activates chain-of-thought reasoning
    )
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens     = max_new_tokens,
            do_sample          = True,
            temperature        = 0.6,   # Qwen3 recommended for thinking mode
            top_p              = 0.95,
            top_k              = 20,
            repetition_penalty = 1.1,
        )

    full   = tokenizer.decode(out[0][inputs["input_ids"].shape[1]:],
                               skip_special_tokens=True).strip()
    # Extract thinking block and final answer
    think_match = re.search(r'<think>(.*?)</think>', full, re.DOTALL)
    thinking    = think_match.group(1).strip() if think_match else ""
    answer      = re.sub(r'<think>.*?</think>', '', full, flags=re.DOTALL).strip()
    return thinking, answer


test_q = "What is the relationship between order and chaos in human experience?"
print(f"Q: {test_q}")
print()
thinking, answer = ask_v2_thinking(test_q)
if thinking:
    print(f"[Thinking]\n{thinking[:400]}{'…' if len(thinking)>400 else ''}\n")
print(f"[Answer]\n{answer}")

---
## Conclusions

### What Changed and Why

Three targeted improvements addressed the root causes identified in the V1
post-mortem:

**1. Synthetic Q&A training data** resolved the passage-regurgitation problem.
By training on `(question, Peterson passage)` pairs instead of
`(passage fragment, passage continuation)` pairs, the model learned to answer
questions in Peterson's voice rather than to continue book text.  This is the
single highest-leverage change.

**2. Three training epochs** moved the model out of the memorisation phase and
into genuine stylistic generalisation.  After three full passes over the
~2,400 Q&A examples the model has learned to apply Peterson's vocabulary,
framing, and rhetorical style to questions it has never seen during training.

**3. LoRA rank r=32** doubled the adapter's representational capacity, giving
it enough independent directions to simultaneously encode Peterson's thematic
vocabulary, sentence rhythm, and conceptual framework — aspects of style that
compete for the limited parameter budget at r=16.

### Running the Comparison

To evaluate V2 against V1 and both base models, update the all-models
comparison notebook:

1. Delete (or rename) `comparison_cache_all_models/qwen3_tuned_results.pkl`
   so the comparison notebook re-runs inference for the fine-tuned Qwen3 model
2. Update `QWEN3_LORA_PATH` in the comparison notebook to point to
   `./outputs/qwen3_14b_peterson_v2_lora/`
3. Re-run `AllModels_JordanPeterson_Comparison.ipynb`

### Paths to Further Improvement

If V2 still shows limitations, the next levers in order of expected impact:

| Option | Expected gain | Effort |
|--------|--------------|--------|
| 5 epochs (instead of 3) | Marginal style improvement; risk of overfitting | Low |
| Larger Q&A dataset (3 questions/passage) | More data diversity | Medium |
| `r=64, alpha=64` | More adapter capacity; +200 MB checkpoint | Low |
| Supervised answers (human-edited responses) | Highest quality ceiling | High |
| Qwen3-32B (if it fits in 24 GB) | Stronger base model | High |